<a href="https://colab.research.google.com/github/AmbiguousError/orca_watch/blob/main/devonport_orca_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install libraries required

In [ ]:
!pip install yt-dlp ultralytics opencv-python matplotlib



YOLOv8 Model: Download a pre-trained YOLOv8 object detection model.
Deets here: https://github.com/ultralytics/ultralytics



In [ ]:
! yolo task=detect mode=predict model=yolov8n.pt


check feed formats available

In [ ]:
!yt-dlp -F https://www.youtube.com/watch?v=6VHnY6ixjLc


run script, will apparently show pics of orca in cell when viewed ... should setup test

In [ ]:
import cv2
import yt_dlp
import subprocess
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt

def detect_orcas(video_filename, model_path="yolov8n.pt"):
    """Detects orcas in a video file."""
    model = YOLO(model_path)
    cap = cv2.VideoCapture(video_filename)
    while cap.isOpened():
        success, frame = cap.read()
        if success:
            results = model(frame)
            for result in results:
                for box in result.boxes:
                    if result.names[box.cls[0]] == "orca":
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        label = f"{result.names[box.cls[0]]} {box.conf[0]:.2f}"
                        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Display the frame using Matplotlib
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # Convert to RGB for Matplotlib
            plt.title("Orca Detection")
            plt.show()

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

def process_live_stream(live_stream_url, model_path="yolov8n.pt"):
    """Processes a live stream in chunks."""
    temp_filename = "temp.ts"  # Change to .ts for MPEG-TS
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        'verbose': True,
        'noplaylist': True,
        'merge_output_format': 'mp4'  # Merge audio and video into mp4 after download
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(live_stream_url)
        video_filename = ydl.prepare_filename(info)

        while True:
            # Record a segment of the stream
            ffmpeg_cmd = f"ffmpeg -protocol_whitelist file,http,https,tcp,tls,crypto -i {video_filename} -c copy -bsf:a aac_adtstoasc -t 30 {temp_filename}"
            print(f"Recording segment: {ffmpeg_cmd}")

            try:
                subprocess.run(ffmpeg_cmd, shell=True, check=True)
            except subprocess.CalledProcessError as e:
                print(f"FFmpeg error: {e.stderr}")  # Print FFmpeg error
                break

            if os.path.exists(temp_filename):
                print(f"Processing segment: {temp_filename}")
                detect_orcas(temp_filename, model_path)
                os.remove(temp_filename)  # Remove the temporary file
            else:
                print("Error: Temporary file not created.")


if __name__ == "__main__":
    youtube_url = "https://www.youtube.com/watch?v=6VHnY6ixjLc"  # Replace with the actual live stream URL
    process_live_stream(youtube_url)
